[![imagenes/pythonista.png](imagenes/pythonista.png)](https://pythonista.io)

# Gestión de *URLs* y funciones de vista.

**ADVERTENCIA:** Es necesario haber creado previamente el proyecto definido en la notebook [```02_introduccion_a_django.ipynb```](02_introduccion_a_django.ipynb) localizado en el directorio [```tutorial```](tutorial).

In [ ]:
! tree tutorial

## *URLs* y funciones de vista.

*Django* tiene la capacidad de identificar patrones específicos a partir de la *URL* a la que pretende acceder un cliente, de tal manera que puede ligar una "función de vista" a dicho patrón.

### Los scripts ```urls.py```.

Tanto el directorio de un proyecto como el directorio de una aplicación pueden contener un archivo ```urls.py```.

Estos scripts son usados por *Django* para relacionar una *URL* con un objeto invocable (funciones o métodos) que será referido como una *función de vista*.

## El script ```urls.py``` de un proyecto.

Los proyectos de *Django* siempre deben de hacer referencia a un archivo ```urls.py``` el cual se importaría como un módulo y a partir del cual pueden ser mapeados todas las *URLs* del proyecto y de sus aplicaciones.

La localización de este archivo es definda con la variable ```ROOT_URLCONF``` del archivo ```settings.py``` del proyecto.

**Ejemplo:**

* En caso de haber ejecutado correctamente todas las celdas de la notebook [```02_introduccion_a_django.ipynb```](02_introduccion_a_django.ipynb), la variable ```settings.ROOT_URLCONF``` indicaría que el script  ```'tutorial.urls'```

In [ ]:
import tutorial.tutorial.settings as settings

In [ ]:
settings.ROOT_URLCONF

* El archivo se localizaría entonces en ```tutorial/tutorial/urls.py``` y tendría el siguiente contenido (sin incluir docstrings): 

``` python
from django.contrib import admin
from django.urls import path

urlpatterns = [
    path('admin/', admin.site.urls),
]
```

In [ ]:
%pycat tutorial/tutorial/urls.py

* Este archivo realiza lo siguiente:

    * Importa la función ```django.contrib.admin``` y le asigna el nombre ```admin```.
    * Importa la función ```django.urls.path``` y le asigna el nombre ```path```.
    * Define al objeto tipo ```list``` con el nombre ```urlpatterns```.

### El paquete ```django.contrib```.

Este paquete contiene una biblioteca general herramientas de *Django*. 

La documentación de este paquete puede ser consultada en https://docs.djangoproject.com/en/2.2/ref/contrib/

In [ ]:
import django.contrib

In [ ]:
help(django.contrib)

#### El paquete ```django.contrib.admin```.

Este paquete permite desplegar una aplicación que contiene al sitio de administración del proyecto. 

La documentación de este paquete puede ser consultada en https://docs.djangoproject.com/en/2.2/ref/contrib/admin/

**Notas:**
* Aún cuando este paquete es cargado por defecto al crear un proyecto de *Django*, puede ser un recurso opcional.
* El sitio de administración requiere que se definan usuarios con los roles y permisos correspondientes. Ese tema se verá en capítulos posteriores.

### El paquete ```django.urls```.

Este paquete contiene a las herramientas que permitirán identificar patrones de *URLs* específicas y ligarlas con  funciones de vista.

In [ ]:
import django.urls

In [ ]:
help(django.urls)

#### La función ```django.urls.path()```.

Esta función permite relacionar a un patrón de *URL* a una función.

```
django.urls.path('<patrón>', <función>, kwargs, name='<nombre>')
```
Donde:

* ```patrón de ruta``` corresponde a un patrón de *URL*.
* ```función``` corresponde una función. 
* ```nombre``` es el nombre que se le dará a esta relación (opcional).


#### La función ```django.urls.include()```.

Por convención, se utliza el nombre ```include``` cuando se desea importar algún archivo como un módulo de forma similar a la palabra reservada de Python ```import```. 

En este caso, la función ```django.urls.include()``` permite cargar un archivo con una estructura de un archivo ```urls.py``` localizado en otra ruta.

``` python
django.urls.include('<patrón>', include('<módulo>')
```

Donde:

* ```patrón de ruta``` corresponde a un patrón de *URL*.
* ```módulo``` corresponde un archivo que contenga una estructura ```urls.py``` expresado como módulo. 

### La lista ```urlpatterns```.

El nombre ```urlpatterns``` corresponde a un objeto de tipo ```list```, el cual puede contener una colección de funciones ```django.urls.path()```.

### Ejemplo ilustrativo.

El archivo ```src/06/tutorial_urls.py``` contiene el siguiente código:

``` python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [path('admin/', admin.site.urls), 
               path('main/', include('main.urls')),]
```

* El primer elemento de ```urlpatterns``` relaciona al patron de  *URL* ```admin/``` con la función ```admin.site.urls()```.
* El segundo elemento de ```urlpatterns``` indica que es necesario importar el módulo ```main.urls``` para definir todas las *URLs* relativas a la ruta```main/```.

* La siguiente celda sustituirá al archivo ```tutorial/tutorial/urls.py``` con el archivo ```src/tutorial_urls.py```.

In [ ]:
%cp src/06/tutorial_urls.py tutorial/tutorial/urls.py

In [ ]:
%pycat tutorial/tutorial/urls.py

En vista de que ```tutorial/tutorial/urls.py``` cargará como módulo el contenido de 
```tutorial/main/urls.py``` es necesario que el archivo de referencia exista.

El archivo ```src/06/main_urls.py``` contiene el siguiente código:

``` python
from django.urls import path
from . import views

urlpatterns = [path('', views.index, name="inicio"), 
               path('vista', views.vista)]
```

Este código indica lo siguiente: 

* Importar la función ```django.urls.path()```.
* Importar el módulo ```views``` localizado en el mismo directorio donde se encuentra el archivo ```tutorial/main/urls.py``` (denotado por el punto ```.```). 
* La lista ```urlpatterns``` definen dos patrones de rutas:
   * La ruta de ```main/```, la cual se denota por una cadena de caracteres vacía ```''```, ligándola a la función ```views.index()```.
   * La ruta de ```main/vista```, la cual se denota por una cadena de caracteres ```'vista'```, ligándola a la función ```views.vista()```.

* La siguiente celda copiará el archivo ```src/06/main_urls.py``` a ```tutorial/main/urls.py```.

In [ ]:
%cp src/06/main_urls.py tutorial/main/urls.py

In [ ]:
%pycat tutorial/main/urls.py

## Las funciones de vista.

Una función de vista es aquella que le permite al servidor de *Django* enviar una respuesta (*response*) a partir de una petición *request* hecha por un cliente a la *URL* correspondiente. 

Una vez que un cliente envía una petición a una *URL* gestionada por el servidor de *Django* ocurre lo siguiente:

* El servidor verifica que la *URL* coincida con alguno de los patrones definidos en los archivos ```urls.py```. 
* En caso de que exista una coincidencia, se ejecutará la función de vista correspondiente de la siguiente forma:
    * Ingresando un objeto instanciado de ```django.http.HttpRequest``` como argumento. 
    * La función deberá de regresar un objeto instanciado de alguna subclase de ```django.http.HttpResponse```. Dicho objeto será utilizado como base para que el servidor transmita una respuesta al cliente que hizo la petición.
* En caso de que no exista coincidencia, se gestionará un error de tipo ```404```.


**Nota:** Tanto las clases ```django.http.HttpRequest``` y ```django.http.HttpResponse```, así como la gestión de errores se estudiarán en capítulos subsecuentes.

## Los scripts *views.py*.

*Django* define en su estructura de archivos ```views.py```, los cuales contienen a las funciones de vista de las aplicaciones. 

**Ejemplo:**

El archivo ```tutorial/main/views.py``` es referido como un módulo en el archivo ```tutorial/main/urls.py```.

Hasta el momento, este archivo no contiene ninguna función.

In [ ]:
%pycat tutorial/main/views.py

### Ejemplo ilustrativo.

El archivo ```src/06/views.py``` contiene lo siguiente:

``` python
from django.http import HttpResponse

# Create your views here.
def index(request):
    return HttpResponse("<h1>Hola, mundo.</h1>")

def vista(request):
    pass
```

Este código realiza lo siguiente:

* Importa la clase ```HttpResponse``` desde el módulo ```django.http```.
* Define la función de vista ```index()```, la cual regresa una instancia de ```HttpResponse``` conteniendo código en *HTML*.
* Define la función de vista ```vista()```, la cual regresa ```None```.


* La siguiente celda sustituirá al archivo ```tutorial/main/views.py``` con el archivo ```src/06/views.py```.

In [ ]:
%cp src/06/views.py tutorial/main/views.py

In [ ]:
%pycat tutorial/main/views.py

* Las siguientes celda iniciará el servidor de *Django* del proyecto ```tutorial```.

In [ ]:
%cd tutorial

**ADVERTENCIAS:** 

* Al ejecutar la siguiente celda el servidor se inciará desde la notebook, por lo que para ejecutar cualquier otra celda es necesario interrumpir la ejecución del kernel.

* Asegúrese que no haya otro servicio escuchando en el puerto ```5000```.

In [ ]:
! ./manage.py runserver 0.0.0.0:5000

* Al acceder a [```http://localhost:5000/admin```](http://localhost:5000/admin) aparecerá la interfaz de administración del proyecto.
* Al acceder a [```http://localhost:5000/main/```](http://localhost:5000/main/) aparecerá un mensaje de encabezado desplegando ```Hola, mundo.```.
* Al acceder a [```http://localhost:5000/main/vista```](http://localhost:5000/main/vista) se mostrará un mensaje de error. Debido a que *Django* está configurado en modo de depuración, desplegará los detalles del error de ejecución.
* Al acceder a cualquier otra *URL* distinta a las indicadas, se mostrará un mensaje de error ```404```. Debido a que *Django* está configurado en modo de depuracion, desplegará los detalles y los patrones de ```URL``` válidos.

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2019.</p>